In [ ]:
import numpy as np

In [ ]:
'''Definimos o problema CSP com uma classe reduzida, há mais métodos importantes que podem 
ser utilizados porém por enquanto acho que com os 2 (atribuir e n_conflitos) podemos resolver 
o problema'''
class CSP():

    def __init__(self, variaveis, dominios, vizinhos, restricoes, dim):

        # Lista com as variáveis utilizadas
        self.variaveis = variaveis
        # Dict com todos os possíveis valores para cada variável
        self.dominios = dominios
        # Dict indicando quais variáveis possuem restrições com quais
        self.vizinhos = vizinhos
        # Função que irá checar se há variáveis descumprindo restrições
        self.restricoes = restricoes
        # Manter a dimensão do problema
        self.dim = dim

    '''Iremos passar os parâmetros e o método irá atualizar o valor da variável
    que estará no dict "atribuições" junto com seu valor'''
    def atribuir(self, var, val, atribuicoes):

        self.dominios.append(atribuicoes[var])
        self.dominios.remove(val)
        atribuicoes[var] = val


    '''Irá retornar a quantia de variáveis em conflito e identificando quais são 
    as variáveis'''
    def n_conflitos(self, var, val, atribuicoes):

        conflitos = 0
        var_conflitos = []

        def conflito(var2):
            return self.restricoes(self.dim, var, val, var2, atribuicoes)

        for v in self.vizinhos:
            if conflito(v) == 1:
                conflitos+=1
                var_conflitos.append(v)
            
        return [conflitos, var_conflitos]

In [ ]:
'''Classe que define o problema, já usada para resolver o problema das 8-rainhas 
usando busca local (aqui está menor por conta de o problema ser resolvido inteiramente
em CSP, estamos à utilizando pois já estava pronta)'''
class oitorainhas:

    def __init__(self, x, y, tabuleiro=np.array([None])):
        self.x = x
        self.y = y

        if tabuleiro.all() == None:
            self.tabuleiro = np.zeros([x,y])
            rainhas = 0
            while True:

                xx =  np.random.randint(self.x)
                yy =  np.random.randint(self.y)

                if self.tabuleiro[xx][yy] != 1:
                    self.tabuleiro[xx][yy] = 1
                    rainhas+=1
                    
                if rainhas ==  self.x:
                    break
        else:
            self.tabuleiro = tabuleiro
    
    def test_objetivo(self, tabuleiro):

        invalido = 0

        for icol in range(self.y):

            for irow in range(self.x):
    
                if(tabuleiro[irow][icol] == 1):
               

                    # Contar 1s na coluna icol
                    for k in range(self.y):
                        if (tabuleiro[irow][k] == 1 and [irow, icol]!=[irow, k]):
                            invalido+=1

                    # Contar 1s na fileira irow
                    for k in range(self.x):
                        if (tabuleiro[k][icol] == 1 and [irow, icol]!=[k, icol]):
                            invalido+=1

                    # encontrar diagonal esquerda (secundaria)
                    col_esq = icol
                    row_esq = irow
                    while(col_esq != 0 and row_esq != (self.x-1)):
                        col_esq-=1
                        row_esq+=1       

                    # cencontrar diagonal direita (primaria)
                    col_dir = icol
                    row_dir = irow
                    while(col_dir != (self.y-1) and row_dir != (self.x-1)):
                        col_dir+=1
                        row_dir+=1

                    # contar 1's diagonal esquerda (secundaria)
                    while(row_esq >= 0 and col_esq <= self.y-1):

                        if (tabuleiro[row_esq][col_esq] == 1 and [irow, icol]!=[row_esq, col_esq]):
                            invalido+=1

                        col_esq+=1
                        row_esq-=1

                    # contar 1's diagonal direita (primaria)
                    while(col_dir>=0 and row_dir>=0 and [irow, icol]!=[row_dir, col_dir]):

                        if (tabuleiro[row_dir][col_dir] == 1):
                            invalido+=1
                            
                        col_dir-=1
                        row_dir-=1
       
        if invalido == 0:
            return True
        else:
            return False

In [ ]:
'''Função que aponta as restrições descumpridas, sendo usada em "restricoes" na classe CSP'''
def restricao_rainha(shape, var, coor, var2, atribu):

    valores = []

    coor1 = coor[0]
    coor2 = coor[1] - 1
    # Lateral Esquerda
    while coor2>=0:

        valores.append([coor1, coor2])
        coor2-=1

    coor1 = coor[0]
    coor2 = coor[1] + 1
    # Lateral Direita
    while coor2<shape[1]:

        valores.append([coor1, coor2])
        coor2+=1

    coor1 = coor[0] + 1
    coor2 = coor[1]
    # Baixo
    while coor1<shape[0]:

        valores.append([coor1, coor2])
        coor1+=1

    coor1 = coor[0] - 1
    coor2 = coor[1] 
    # Cima
    while coor1>=0:

        valores.append([coor1, coor2])
        coor1-=1

    coor1 = coor[0] + 1
    coor2 = coor[1] + 1
    # Diagonal Inferior Direita
    while coor1<shape[0] and coor2<shape[1]:

        valores.append([coor1, coor2])
        coor1+=1
        coor2+=1

    coor1 = coor[0] - 1
    coor2 = coor[1] - 1
    # Diagonal Superio Esquerda
    while coor1>=0 and coor2>=0:

        valores.append([coor1, coor2])
        coor1-=1
        coor2-=1
    
    coor1 = coor[0] + 1
    coor2 = coor[1] - 1
    # Diagonal Inferio Esquerda
    while coor1<shape[0] and coor2>=0:

        valores.append([coor1, coor2])
        coor1+=1
        coor2-=1

    coor1 = coor[0] - 1
    coor2 = coor[1] + 1
    # Diagonal Superior Direita
    while coor1>=0 and coor2<shape[1]:

        valores.append([coor1, coor2])
        coor1-=1
        coor2+=1


    if atribu[var2] in valores:
        # Existe conflito entre variáveis
        return 1
    return 0

In [ ]:
'''Algoritmo utilizado para resolver o desafio das oito rainhas usando CSP'''
def csp_rainhas(problema, concluido, atribui):
    var = 0
    busca = True
    ll=0

    while busca:
        ll+=1

        # Manter uma flag para parar o loop interno caso ache valor para variável
        var_placed = True
        # Lista com a localização das possíveis ações
        ch = list(range(len(problema.dominios)))

        while var_placed:
    
            
            if len(ch) != 0:
                # Escolher de forma aleatória a posição do próximo passo
                choice = np.random.choice(ch)
                # Excluir da lista a ação já selecionada
                ch.remove(choice)

            # Caso as possíveis ações tenham acabado sem ação válida para a váriavel
            else:

                conflitos = problema.n_conflitos(var, value, atribui)[1]
                choice = np.random.choice(range(len(conflitos)))
                var = conflitos[choice]

                # Recolocar as variáveis que participam da restrição como não postas nas notações
                for j in conflitos:
                    concluido[j] = "False"

                break

            # Selecionando o passo
            value = problema.dominios[choice]
            # Confirmando no registro atribui o passo
            problema.atribuir(var, value, atribui)

            # Caso o passo não tenha causado conflitos
            if problema.n_conflitos(var, value, atribui)[0] == 0:
                
                # Setar nas anotações como concluido
                concluido[var] = "Concluido"

                # Selecionar todas as variáveis em anotações que ainda não estão postas
                varr = []
                for i in range(len(concluido.keys())):
                    if concluido[i] == "False":
                        varr.append(i)

                # Escolher uma das variáveis 
                if len(varr) >= 1:
                    var = np.random.choice(varr)
                else:
                    busca=False
                    print(ll)

                var_placed = False

In [ ]:
'''Irá retornar a localização das rainhas(elas são iniciadas em posições aleatórias)
e todos os locais possíveis para movimentação '''
def encontrar_rainhas(tabuleiro):
    x, y = tabuleiro.shape
    id = 0
    rainhas = {}
    dominio = []

    for i in range(x):
        for j in range(y):
            if(tabuleiro[i, j] == 1):
                rainhas[id] = [i,j]
                id+=1
            else:
                dominio.append([i, j])

    return rainhas, dominio

rainhas = oitorainhas(8,8)

atribui, dominio = encontrar_rainhas(rainhas.tabuleiro)
var = list(range(8))
# Todas as variáveis são vizinhas(possuem restrições entre si)
vizinhos = {}
for i in range(8):
    aux = list(range(8))
    aux.remove(i)
    vizinhos[i] = aux
# Aqui será mantido se uma variável foi ou não colocada respeitando restrições
concluido = {}
for i in range(8):
    concluido[i] = 'False'

In [ ]:
problema = CSP(var, dominio, vizinhos, restricao_rainha, [8,8])
csp_rainhas(problema, concluido, atribui)

5285


In [ ]:
'''Reconstruindo o tabuleiro com as posições corretas'''
tabuleiro = np.zeros([8,8])       

for i in range(len(atribui.values())):
    tabuleiro[list(atribui.values())[i][0], list(atribui.values())[i][1]] = 1

rainhas = oitorainhas(8,8,tabuleiro)
rainhas.tabuleiro

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.]])